In [ ]:
def improving_manual_icp(list_stitches,
                         list_cool_transformations,
                         results_improving_icp,
                         downsample_voxel_size = 0.1,
                         voxel_size=0.1,
                         
                         visualization_on = False,
                         print_statements = True
                        ):
    
    # initialization 
    source = copy.deepcopy(list_stitches[0])
    target =  copy.deepcopy(list_stitches[1])
    print(f"{lables[0]} {list_stitches[0]}" )
    print(f"{lables[1]} {list_stitches[1]}" )

    #downsampling
    source = source.voxel_down_sample(voxel_size=downsample_voxel_size)
    target = target.voxel_down_sample(voxel_size=downsample_voxel_size)
    print (source)
    print(target)

    if visualization_on == True:
        custom_draw_geometry(source+target,
                         mytitle = "initial",
                         #params = myparams,  
                         #configuration_file = myconfiguration_file, 
                         take_screen_shot = True,
                         rotate = False,
                         #onewindow = True
                        )

    
    # prepare data set 
    new_source, new_target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(source,
                                                                                                     target,
                                                                                                     voxel_size=voxel_size, 
                                                                                                     mytitle ="",
                                                                                                     print_statements = print_statements
                                                                                             )
    # run icp
    result_icp = refine_registration(new_source, new_target, 
                                                     source_fpfh, target_fpfh,
                                                     voxel_size=voxel_size,
                                                     mytranformation =list_cool_transformations[-1],
                                                     print_statements = print_statements
                                                    )

    overlapping_points = result_icp.correspondence_set

    get_num_points([new_source, new_target],True)

    print("Transformation is:")
    print(result_icp.transformation)
    print ("overlapping points : " ,len(np.asarray(overlapping_points)))
    print (result_icp)

    #draw_registration_result(new_source, new_target, 
                                     #result_icp.transformation,
                                     #title = "after_optimized_icp_registration"
                                     #)
    
    #build the final newpointcloud and eventually save it
    newpointcloud = save_registration_result(source, target, result_icp.transformation, 
                                             title = "after_optimized_icp_registration", 
                                             save_result = False,
                                             visualize_result = True)

    list_cool_transformations.append(result_icp.transformation)
    results_improving_icp.append(result_icp)
    

## manual registration icp

In [1]:
def evaluation_by_manual_registration(source,target,
                            
                             threshold = 0.03,
                             picked_id_source = None,
                             picked_id_target = None,
                                      
                             #statements
                             print_statements = False,
                             visualization_on = False
                             ):
    """
    transf,picked_id_source,picked_id_target = evaluation_by_manual_registration
    
    """
    
    

    if picked_id_source is None: 
        # pick points from two point clouds and builds correspondences
        picked_id_source = pick_points(source)
        
    if picked_id_target is None:
        picked_id_target = pick_points(target)
    
    print (picked_id_source)
    assert (len(picked_id_source) >= 3 and len(picked_id_target) >= 3)
    assert (len(picked_id_source) == len(picked_id_target))
    corr = np.zeros((len(picked_id_source), 2))
    corr[:, 0] = picked_id_source
    corr[:, 1] = picked_id_target

    # estimate rough transformation using correspondences
    if print_statements == True:
        print("Compute a rough transform using the correspondences given by user")
    p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    trans_init = p2p.compute_transformation(source, target,
                                            o3d.utility.Vector2iVector(corr))

    # point-to-point ICP for refinement
    if print_statements == True:
        print("Perform point-to-point ICP refinement")
    threshold = threshold  #  distance threshold
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    transf = reg_p2p.transformation 
    
    print (reg_p2p)
    if visualization_on == True: 
        draw_registration_result(source, target, transf)
        
    if print_statements == True:
        print("")
        print (reg_p2p)
        print(transf) 
    return transf,picked_id_source,picked_id_target,reg_p2p

In [ ]:
def full_evaluation_pipeline(
                             #input 
                             list_stitches,
                             labels_stitches = ["model_pc","stitched_pc"],
                             color_stitches = [[1, 0.706, 0],[0, 0.651, 0.929]],
                             
                             #plotting 
                             params = myparams,  # parameter for camera point view, json file via pressing P
                             configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                             take_screen_shot = True,
                             rotate = False,
                             onewindow = True,
                             
                             #scaling
                             scaling_needed = True,
                             scale_factor = 1000,
    
                             #downsampling
                             voxels = [0.1,0.2],
                             
                             #evaluation 
                             threshold = 0.03,
                             picked_id_source = None,
                             picked_id_target = None,
    
                             #statements
                             print_statements = False,
                             visualization_on = False
                            ):
    """
    list_stitches = [pcd,st_pcd]
        pcd = original point cloud of the model
        st_pcd = our stitched point cloud that we want to evaluate against the model 
    
    
    """
    # check input size
    assert len(list_stitches) == len(labels_stitches) 
    assert len(labels_stitches) == len(color_stitches)
    assert len(color_stitches) == 2
    
    #
    pcd,st_pcd = list_stitches

    ## assign colors
    list_stitches[0].paint_uniform_color(color_stitches[0])
    list_stitches[1].paint_uniform_color(color_stitches[1])
    
    # mytitle
    mytitle = f"{labels_stitches[0]}_{get_color_name(color_stitches[0])}-{labels_stitches[1]}_{get_color_name(color_stitches[1])}"
    
    
    if visualization_on == True:
        # plot a list of geometries
        custom_draw_geometry(list_stitches[0]+list_stitches[1],
                             mytitle = "initial_"+mytitle,
                             params = params,  # parameter for camera point view, json file via pressing P
                             configuration_file = configuration_file, # configuration file for properties, json file via pressing o
                             take_screen_shot = take_screen_shot,
                             rotate = rotate,
                             onewindow = onewindow)
    
    temp_pcd = copy.deepcopy(pcd)
    if scaling_needed == True: 
        #define scaling function 
        trans_scale = np.asarray([[scale_factor, 0.0, 0.0, 0.0], 
                                 [0.0, scale_factor, 0.0, 0.0],
                                 [0.0, 0.0, scale_factor, 0.0], 
                                 [0.0, 0.0, 0.0, 1.0]])

        # apply scaling function
        #temp_pcd = copy.deepcopy(pcd)
        temp_pcd.transform(trans_scale)
        list_stitches[0] = temp_pcd

        if visualization_on == True:
            # plot a list of geometries
            custom_draw_geometry(list_stitches[0]+list_stitches[1],
                                 mytitle = mytitle+"after_resizing",
                                 params = params,  # parameter for camera point view, json file via pressing P
                                 configuration_file = configuration_file, # configuration file for properties, json file via pressing o
                                 take_screen_shot = take_screen_shot,
                                 rotate = rotate,
                                 onewindow = onewindow)
        
    #downsampling
    down_pcd = temp_pcd.voxel_down_sample(voxel_size=voxels[0])
    down_st_pcd = st_pcd.voxel_down_sample(voxel_size=voxels[1])
    downsampled_stitches = [down_pcd,down_st_pcd]
    
    ## for some reason this kills the kernel??
    ## TODO Check why on hearth
    #if print_statements == True:
        #print (len(np.asarray(down_pcd.points)))
        #print (len(np.asarray(down_st_pcd.points)))
        
    #set source and target
    source = copy.deepcopy(downsampled_stitches[0])
    target = copy.deepcopy(downsampled_stitches[1])
    
    # evaluation by manual icp registration 
    ## TO DO check here
    transf,new_picked_id_source,new_picked_id_target, registration = evaluation_by_manual_registration(source,target,
                                                                             threshold = 0.03,
                                                                             picked_id_source = picked_id_source,
                                                                             picked_id_target = picked_id_target,
                                                                             print_statements = print_statements,
                                                                             visualization_on = visualization_on
                                                                            )
    
    return transf,new_picked_id_source,new_picked_id_target,registration,list_stitches

## with recording

for some reason it kills my kernel..

In [2]:
import time

In [ ]:
def evaluation_by_manual_registration_with_recording(source,target,
                            
                             threshold = 0.03,
                             picked_id_source = None,
                             picked_id_target = None,
                             icp_iteration = 100,
                                      
                             #statements
                             print_statements = False,
                             visualization_on = False,
                             save_image = False
                             ):
    """
    transf,picked_id_source,picked_id_target = evaluation_by_manual_registration
    
    """
    
    

    if picked_id_source is None: 
        # pick points from two point clouds and builds correspondences
        picked_id_source = pick_points(source)
        
    if picked_id_target is None:
        picked_id_target = pick_points(target)
    
    print (picked_id_source)
    assert (len(picked_id_source) >= 3 and len(picked_id_target) >= 3)
    assert (len(picked_id_source) == len(picked_id_target))
    corr = np.zeros((len(picked_id_source), 2))
    corr[:, 0] = picked_id_source
    corr[:, 1] = picked_id_target

    # estimate rough transformation using correspondences
    if print_statements == True:
        print("Compute a rough transform using the correspondences given by user")
    p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    trans_init = p2p.compute_transformation(source, target,
                                            o3d.utility.Vector2iVector(corr))

    # define transf list
    transf_list = []
    transf_list.append(trans_init)
    
    
    # point-to-point ICP for refinement
    if print_statements == True:
        print("Perform point-to-point ICP refinement")
    threshold = threshold  #  distance threshold
    
    
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(source)
    vis.add_geometry(target)
    # perform point to plane ICP refinement
    for i in range(icp_iteration):
        time.sleep(1)    
        reg_p2p = o3d.pipelines.registration.registration_icp(
            source, target, threshold, transf_list[-1],
            o3d.pipelines.registration.TransformationEstimationPointToPoint())
        transf = reg_p2p.transformation 
        
        transf_list.append(transf)
        source.transform(transf)
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        if save_image:
            vis.capture_screen_image("temp_%04d.jpg" % i)
            
    vis.destroy_window()
    
    print (reg_p2p)
    if visualization_on == True: 
        draw_registration_result(source, target, transf)
        
    if print_statements == True:
        print("")
        print (reg_p2p)
        print(transf) 
    return transf,picked_id_source,picked_id_target,reg_p2p

In [ ]:
def full_evaluation_pipeline_with_recording(
                             #input 
                             list_stitches,
                             labels_stitches = ["model_pc","stitched_pc"],
                             color_stitches = [[1, 0.706, 0],[0, 0.651, 0.929]],
                             
                             #plotting 
                             params = myparams,  # parameter for camera point view, json file via pressing P
                             configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                             take_screen_shot = True,
                             rotate = False,
                             onewindow = True,
                             
                             #scaling
                             scaling_needed = False,
                             scale_factor = 1000,
    
                             #downsampling
                             voxels = [0.1,0.2],
                             
                             #evaluation 
                             threshold = 0.03,
                             picked_id_source = None,
                             picked_id_target = None,
    
                             #statements
                             print_statements = False,
                             visualization_on = False
                            ):
    """
    list_stitches = [pcd,st_pcd]
        pcd = original point cloud of the model
        st_pcd = our stitched point cloud that we want to evaluate against the model 
    
    
    """
    # check input size
    assert len(list_stitches) == len(labels_stitches) 
    assert len(labels_stitches) == len(color_stitches)
    assert len(color_stitches) == 2
    
    #
    pcd,st_pcd = list_stitches

    ## assign colors
    list_stitches[0].paint_uniform_color(color_stitches[0])
    list_stitches[1].paint_uniform_color(color_stitches[1])
    
    # mytitle
    mytitle = f"{labels_stitches[0]}_{get_color_name(color_stitches[0])}-{labels_stitches[1]}_{get_color_name(color_stitches[1])}"
    
    
    if visualization_on == True:
        # plot a list of geometries
        custom_draw_geometry(list_stitches[0]+list_stitches[1],
                             mytitle = "initial_"+mytitle,
                             params = params,  # parameter for camera point view, json file via pressing P
                             configuration_file = configuration_file, # configuration file for properties, json file via pressing o
                             take_screen_shot = take_screen_shot,
                             rotate = rotate,
                             onewindow = onewindow)
    
    temp_pcd = copy.deepcopy(pcd)
    if scaling_needed == True: 
        #define scaling function 
        trans_scale = np.asarray([[scale_factor, 0.0, 0.0, 0.0], 
                                 [0.0, scale_factor, 0.0, 0.0],
                                 [0.0, 0.0, scale_factor, 0.0], 
                                 [0.0, 0.0, 0.0, 1.0]])

        # apply scaling function
        #temp_pcd = copy.deepcopy(pcd)
        temp_pcd.transform(trans_scale)
        list_stitches[0] = temp_pcd

        if visualization_on == True:
            # plot a list of geometries
            custom_draw_geometry(list_stitches[0]+list_stitches[1],
                                 mytitle = mytitle+"after_resizing",
                                 params = params,  # parameter for camera point view, json file via pressing P
                                 configuration_file = configuration_file, # configuration file for properties, json file via pressing o
                                 take_screen_shot = take_screen_shot,
                                 rotate = rotate,
                                 onewindow = onewindow)
        
    #downsampling
    down_pcd = temp_pcd.voxel_down_sample(voxel_size=voxels[0])
    down_st_pcd = st_pcd.voxel_down_sample(voxel_size=voxels[1])
    downsampled_stitches = [down_pcd,down_st_pcd]
    
    ## for some reason this kills the kernel??
    ## TODO Check why on hearth
    #if print_statements == True:
        #print (len(np.asarray(down_pcd.points)))
        #print (len(np.asarray(down_st_pcd.points)))
        
    #set source and target
    source = copy.deepcopy(downsampled_stitches[0])
    target = copy.deepcopy(downsampled_stitches[1])
    
    # evaluation by manual icp registration 
    ## TO DO check here
    transf,new_picked_id_source,new_picked_id_target, registration = evaluation_by_manual_registration_with_recording(source,target,
                                                                             threshold = 0.03,
                                                                             picked_id_source = picked_id_source,
                                                                             picked_id_target = picked_id_target,
                                                                             print_statements = print_statements,
                                                                             visualization_on = visualization_on
                                                                            )
    
    return transf,new_picked_id_source,new_picked_id_target,registration,list_stitches